In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import plotting

In [ ]:
import scipy
from iminuit import cost
from iminuit import Minuit

In [ ]:
import hist

In [ ]:
plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
df = pd.read_csv("features_CNN_1d_99987.csv")

In [ ]:
X = np.load("images_1d_99987.npy")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y =  df['start_z'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=0
    )

In [ ]:
def predict(X, threshold, threshold_diffs, factor, offset_diffs, offset_even_odd, offset_final):
    threshold_even = threshold
    threshold_odd = threshold
    threshold_diffs_even = threshold_diffs
    threshold_diffs_odd = threshold_diffs
    X_even = X[:,::2]
    X_odd = X[:,1::2]
    diffs_even = np.divide((X_even[:,1:] - X_even[:,:-1]), X_even.max(axis=1).reshape(-1,1), where=X_even.max(axis=1).astype(bool).reshape(-1,1))
    diffs_odd = np.divide((X_odd[:,1:] - X_odd[:,:-1]), X_odd.max(axis=1).reshape(-1,1), where=X_odd.max(axis=1).astype(bool).reshape(-1,1))
    X_even = np.divide(X_even, X_even.max(axis=1).reshape(-1,1), where=X_even.max(axis=1).astype(bool).reshape(-1,1))
    X_odd = np.divide(X_odd, X_odd.max(axis=1).reshape(-1,1), where=X_odd.max(axis=1).astype(bool).reshape(-1,1))
    index_even = np.argmax(X_even > threshold_even, axis=1)
    index_odd = np.argmax(X_odd > threshold_odd, axis=1)
    index_diffs_even = np.argmax(diffs_even > threshold_diffs_even, axis=1)
    index_diffs_odd = np.argmax(diffs_odd > threshold_diffs_odd, axis=1)
    return factor * np.mean([index_even, index_odd + offset_even_odd, index_diffs_even + offset_diffs, index_diffs_odd + offset_even_odd + offset_diffs], axis=0) + offset_final

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from skopt import gp_minimize

In [ ]:
predict(X[:10], 10, 10, 0, 0, 1, 0, 0, 0)

In [ ]:
def evaluate_parameters(params):
    y_pred = predict(X_train, *params)
    return mean_squared_error(y_train, y_pred) 

In [ ]:
gp_res = gp_minimize(evaluate_parameters, [(0.15, 0.3), (2., 3.), (2., 4.), (0., 10.), (-10., 0.), (-260., -225.)], verbose=True, n_calls=100)

In [ ]:
gp_res.x

In [ ]:
y_pred = predict(X_test, *gp_res.x)

In [ ]:
from skopt.plots import plot_convergence

In [ ]:
plot_convergence(gp_res)
plt.title("")
plotting.watermark()
plt.savefig("plots/convergence_classic.png")
plt.savefig("plots/convergence_classic.pdf")

In [ ]:
h = hist.Hist.new.Regular(100, -5, +5, name=r"𝛥z [cm]").Double()

In [ ]:
h.fill(np.squeeze(y_pred) - np.squeeze(y_test))

In [ ]:
entries, edges = h.to_numpy()

In [ ]:
def residual_model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, mu, sigma)

In [ ]:
m = Minuit(cost.BinnedNLL(entries, edges, residual_model), 0, 25)

In [ ]:
res = m.migrad()

In [ ]:
res

In [ ]:
h.plot()
plt.xlabel(r"$\Delta z\;[\mathrm{cm}]$")
plot_range = edges[0], edges[-1]
x = np.linspace(*plot_range, 100)
best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
# best_fit = scipy.stats.norm(0.044, 2.83) # TODO take from fit
n_bins = len(entries)
binsize = (plot_range[1] - plot_range[0]) / n_bins
scale = h.sum() / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0])) * binsize
plt.plot(x, scale * best_fit.pdf(x))
ax = plt.gca()
# plt.text(0.6, 0.9, r"$\mu = 0.044 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.9,
    rf"$\mu = {res.params[0].value:.2f} \pm {res.params[0].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
plt.text(0.01, 0.8, f"Training dataset: {len(y_train)} events\n" f"Test dataset: {len(y_test)} events\n Classical approach", transform=ax.transAxes, usetex=True)
# plt.text(0.6, 0.81, r"$\sigma = 2.83 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.81,
    rf"$\sigma = {res.params[1].value:.3f} \pm {res.params[1].error:.3f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
plotting.watermark()
plt.savefig(f"plots/h_dz_classic_n{len(y)}.pdf")
plt.savefig(f"plots/h_dz_classic_n{len(y)}.png")